<a href="https://colab.research.google.com/github/NgPcAnhh/study/blob/main/whisper(speech_to_text).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

sử dụng ffmpeg

In [ ]:
!apt-get -y install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!ffmpeg -version

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-l

In [ ]:
!ffmpeg -i Practise_recording_20250120_092332.wav -ar 16000 audio.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

sử dụng whisper Ai

In [ ]:
!pip install openai-whisper

In [ ]:
!pip install pydub

In [ ]:
import whisper
import numpy as np
from pydub import AudioSegment
import math
import os

def process_audio_in_chunks(audio_path, model, chunk_duration=30000):
    # Load file âm thanh using pydub
    audio = AudioSegment.from_wav(audio_path)

    # Tính số lượng chunk
    total_duration = len(audio)
    chunks = math.ceil(total_duration / chunk_duration)

    full_transcript = []

    # Xử lý từng chunk
    for i in range(chunks):
        start_time = i * chunk_duration
        end_time = min((i + 1) * chunk_duration, total_duration)

        # Cắt audio chunk
        chunk = audio[start_time:end_time]

        # Tăng âm lượng nếu quá nhỏ
        if chunk.dBFS < -30:
            chunk = chunk + min(20, abs(chunk.dBFS + 20))

        # Lưu chunk tạm thời
        temp_path = f"temp_chunk_{i}.wav"
        chunk.export(temp_path, format="wav")

        # Xử lý với Whisper
        try:
            audio_chunk = whisper.load_audio(temp_path)
            audio_chunk = whisper.pad_or_trim(audio_chunk)
            mel = whisper.log_mel_spectrogram(audio_chunk).to(model.device)

            options = whisper.DecodingOptions(
                language="en",
                fp16=False,
                beam_size=5  # Chỉ sử dụng beam_size
            )

            result = whisper.decode(model, mel, options)

            if result.text.strip():
                print(f"Chunk {i} transcription:", result.text)
                full_transcript.append(result.text)
            else:
                print(f"Chunk {i}: No speech detected")

        except Exception as e:
            print(f"Lỗi khi xử lý chunk {i}: {str(e)}")
            continue
        finally:
            # Xóa file tạm sau khi xử lý xong
            if os.path.exists(temp_path):
                os.remove(temp_path)

    return " ".join(full_transcript)

def main():
    # Tải model
    model = whisper.load_model("base")

    # Đường dẫn file audio
    audio_path = "Practise_recording_20250120_092332.wav"

    # Xử lý audio và in kết quả
    transcript = process_audio_in_chunks(audio_path, model)
    print("\nFull transcription:", transcript)

    if not transcript:
        print("Không phát hiện được giọng nói trong file audio. Vui lòng kiểm tra:")
        print("1. Chất lượng âm thanh đầu vào")
        print("2. Định dạng file audio")
        print("3. Âm lượng của file")

if __name__ == "__main__":
    main()

Chunk 0 transcription: This is the simulated speaking test and I'm your examiner. Can you hear my voice? Okay, please make sure that your microphone is on and keep your surrounding area silent. You should adjust the volume before the test. Are you ready? Okay, let's start. Now, in this first part, I'd like to ask you some questions about yourself.
Chunk 1 transcription: don't like about your area. What are they? For me, there might be a field thing that I don't like about my area, such as the traffic congestion during the big hour, which made commuting super stressful and time-consuming. Now, resolutions can also be
Chunk 2 transcription: And additionally, the lack of green space or public parks in some urban neighborhoods can make it feel credit and less relaxing than the availability of public facilities like libraries or recreational centers might also be limited in some area.
Chunk 3 transcription: Do you know any of your neighbors? Well, I know a few of them. For instance, there's